In [155]:
import pandas as pd
import numpy as np
from embedding import get_embedding

In [156]:
data = {'ESTIMATE': ["I think [AGENT] is a [ROLE]", "[AGENT] must be the [ROLE]", "[AGENT] seems like the [ROLE]", "I'm convinced that [AGENT] is a [ROLE]", "I think [AGENT] is a [ROLE], not a"],
                          'CO': ["I am a [ROLE]", "My role is the [ROLE]", "I must be the [ROLE]", "There are no other [ROLE] besides me"], 
                          'SUSPECT': ["I can't believe [AGENT]", "[AGENT] is suspicious", "I suspect [AGENT] is lying", "I don't trust [AGENT]", "Agent[02] cannot be trusted"], 
                          'VOTE': ["I will vote for [AGENT]"], 
                          'DIVINATION': ["I will divine [AGENT] tonight", "I might investigate [AGENT]"], 
                          'DIVINED': ["I divined [AGENT] as a [TEAM]", "[AGENT] was divined as a [TEAM]", "[AGENT] is determined to be [TEAM]"], 
                          'AGREE': ["I agree with [AGENT]", "I agree with [AGENT]'s opinion", "What [AGENT] said is correct"], 
                          'NOT ESTIMATE': ["I don't think [AGENT] is a [ROLE]", "[AGENT] is not [ROLE]", "[AGENT] doesn't seem like the [ROLE]", "[AGENT] is lying about being a [ROLE]"],
                          'NOT CO': ["I am not a [ROLE]", "It can't be true that I am the [ROLE]"], 
                          'NOT SUSPECT': [ "I don't think [AGENT] is lying", "I trust [AGENT]", '[AGENT] is right'], 
                          'NOT VOTE': ["I won't vote for [AGENT]", "I disagree with voting for [AGENT]"], 
                          'NOT AGREE': ["I disagree with [AGENT]",  "I disagree with [AGENT]'s opinion", "What [AGENT] said is wrong"], 
                          }

In [157]:
df = pd.DataFrame.from_dict(data, orient='index').T # これだとうまくいく
df

,ESTIMATE,CO,SUSPECT,VOTE,DIVINATION,DIVINED,AGREE,NOT ESTIMATE,NOT CO,NOT SUSPECT,NOT VOTE,NOT AGREE
0,I think [AGENT] is a [ROLE],I am a [ROLE],I can't believe [AGENT],I will vote for [AGENT],I will divine [AGENT] tonight,I divined [AGENT] as a [TEAM],I agree with [AGENT],I don't think [AGENT] is a [ROLE],I am not a [ROLE],I don't think [AGENT] is lying,I won't vote for [AGENT],I disagree with [AGENT]
1,[AGENT] must be the [ROLE],My role is the [ROLE],[AGENT] is suspicious,None,I might investigate [AGENT],[AGENT] was divined as a [TEAM],I agree with [AGENT]'s opinion,[AGENT] is not [ROLE],It can't be true that I am the [ROLE],I trust [AGENT],I disagree with voting for [AGENT],I disagree with [AGENT]'s opinion
2,[AGENT] seems like the [ROLE],I must be the [ROLE],I suspect [AGENT] is lying,None,None,[AGENT] is determined to be [TEAM],What [AGENT] said is correct,[AGENT] doesn't seem like the [ROLE],None,[AGENT] is right,None,What [AGENT] said is wrong
3,I'm convinced that [AGENT] is a [ROLE],There are no other [ROLE] besides me,I don't trust [AGENT],None,None,None,None,[AGENT] is lying about being a [ROLE],None,None,None,None
4,None,None,Agent[02] cannot be trusted,None,None,None,None,None,None,None,None,None


In [158]:
df.fillna(np.nan, inplace=True)
df

,ESTIMATE,CO,SUSPECT,VOTE,DIVINATION,DIVINED,AGREE,NOT ESTIMATE,NOT CO,NOT SUSPECT,NOT VOTE,NOT AGREE
0,I think [AGENT] is a [ROLE],I am a [ROLE],I can't believe [AGENT],I will vote for [AGENT],I will divine [AGENT] tonight,I divined [AGENT] as a [TEAM],I agree with [AGENT],I don't think [AGENT] is a [ROLE],I am not a [ROLE],I don't think [AGENT] is lying,I won't vote for [AGENT],I disagree with [AGENT]
1,[AGENT] must be the [ROLE],My role is the [ROLE],[AGENT] is suspicious,NaN,I might investigate [AGENT],[AGENT] was divined as a [TEAM],I agree with [AGENT]'s opinion,[AGENT] is not [ROLE],It can't be true that I am the [ROLE],I trust [AGENT],I disagree with voting for [AGENT],I disagree with [AGENT]'s opinion
2,[AGENT] seems like the [ROLE],I must be the [ROLE],I suspect [AGENT] is lying,NaN,NaN,[AGENT] is determined to be [TEAM],What [AGENT] said is correct,[AGENT] doesn't seem like the [ROLE],NaN,[AGENT] is right,NaN,What [AGENT] said is wrong
3,I'm convinced that [AGENT] is a [ROLE],There are no other [ROLE] besides me,I don't trust [AGENT],NaN,NaN,NaN,NaN,[AGENT] is lying about being a [ROLE],NaN,NaN,NaN,NaN
4,NaN,NaN,Agent[02] cannot be trusted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
class MakeDataframe:
    def __init__(self):
        self.datalist = None

    def make_data(self, df):
        """データフレームを受け取り、[AGENT]と[ROLE]と[TEAM]を全ての組み合わせで埋める"""
        Agents = ["Agent[00]", "Agent[01]", "Agent[02]", "Agent[03]", "Agent[04]"]
        Roles = ["villeger", "seer", "werewolf", "possessed"]
        Teams = ["black", "white", "villeger", "human", "werewolf"] # 精度のためにteamsに"VILLAGER", "WEREWOLF"も追加
        
        self.datalist = []
        for column in df.columns: 
            original_sentences = df[column].dropna().values 
            for original_sentence in original_sentences:
                if "[AGENT]" in original_sentence and "[ROLE]" in original_sentence:
                    for x in Agents:
                        for y in Roles:
                            ans = {} # 各sentenceの情報を入れる辞書
                            ans["sentence"] = original_sentence.replace('[AGENT]', x).replace('[ROLE]', y)
                            ans["Verb"] = column
                            ans["Agent"] = x
                            ans["Role"] = y.upper()
                            ans["Team"] = None
                            self.datalist.append(ans)
                elif "[AGENT]" in original_sentence and "[TEAM]" in original_sentence:
                    for x in Agents:
                        for y in Teams:
                            ans = {} # 各sentenceの情報を入れる辞書
                            ans["sentence"] = original_sentence.replace('[AGENT]', x).replace('[TEAM]', y)
                            ans["Verb"] = column
                            ans["Agent"] = x
                            ans["Role"] = None
                            ans["Team"] = y.upper()
                            self.datalist.append(ans)
                elif "[AGENT]" in original_sentence:
                    for x in Agents:
                        ans = {} # 各sentenceの情報を入れる辞書
                        ans["sentence"] = original_sentence.replace('[AGENT]', x)
                        ans["Verb"] = column
                        ans["Agent"] = x
                        ans["Role"] = None
                        ans["Team"] = None
                        self.datalist.append(ans)
                elif '[ROLE]' in original_sentence:
                    for y in Roles:
                        ans = {} # 各sentenceの情報を入れる辞書
                        ans["sentence"] = original_sentence.replace('[ROLE]', y)
                        ans["Verb"] = column
                        ans["Agent"] = None
                        ans["Role"] = y.upper()
                        ans["Team"] = None
                        self.datalist.append(ans)
        return self.datalist, len(self.datalist)
    def make_dataframe(self, df):
        self.make_data(df)
        df = pd.DataFrame(self.datalist)   
        df["Team"] = df["Team"].replace({"VILLAGER": "WHITE", "HUMAN": "WHITE", "WEREWOLF": "BLACK"})
        return df
    def dataframe_embed(self, df):
        df = self.make_dataframe(df)
        df["embedded"] = df["sentence"].map(lambda x: get_embedding(x))
        # リストに変換
        df["embedded"] = df["embedded"].map(lambda x: list(x))
        df.to_csv("embedded.csv")
        return df
a = MakeDataframe()

In [160]:
a.make_data(df)

([{'sentence': 'I think Agent[00] is a villeger',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'VILLEGER',
   'Team': None},
  {'sentence': 'I think Agent[00] is a seer',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'SEER',
   'Team': None},
  {'sentence': 'I think Agent[00] is a werewolf',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'WEREWOLF',
   'Team': None},
  {'sentence': 'I think Agent[00] is a possessed',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[00]',
   'Role': 'POSSESSED',
   'Team': None},
  {'sentence': 'I think Agent[01] is a villeger',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'VILLEGER',
   'Team': None},
  {'sentence': 'I think Agent[01] is a seer',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'SEER',
   'Team': None},
  {'sentence': 'I think Agent[01] is a werewolf',
   'Verb': 'ESTIMATE',
   'Agent': 'Agent[01]',
   'Role': 'WEREWOLF',
   'Team': None},
  {'sentence': 'I think Agent[01] is a po

In [161]:
a.make_dataframe(df)

,sentence,Verb,Agent,Role,Team
0,I think Agent[00] is a villeger,ESTIMATE,Agent[00],VILLEGER,None
1,I think Agent[00] is a seer,ESTIMATE,Agent[00],SEER,None
2,I think Agent[00] is a werewolf,ESTIMATE,Agent[00],WEREWOLF,None
3,I think Agent[00] is a possessed,ESTIMATE,Agent[00],POSSESSED,None
4,I think Agent[01] is a villeger,ESTIMATE,Agent[01],VILLEGER,None
...,...,...,...,...,...
344,What Agent[00] said is wrong,NOT AGREE,Agent[00],None,None
345,What Agent[01] said is wrong,NOT AGREE,Agent[01],None,None
346,What Agent[02] said is wrong,NOT AGREE,Agent[02],None,None
347,What Agent[03] said is wrong,NOT AGREE,Agent[03],None,None


In [162]:
a.dataframe_embed(df)

,sentence,Verb,Agent,Role,Team,embedded
0,I think Agent[00] is a villeger,ESTIMATE,Agent[00],VILLEGER,None,"[-0.034499943256378174, -0.006359028164297342,..."
1,I think Agent[00] is a seer,ESTIMATE,Agent[00],SEER,None,"[-0.02030394971370697, -0.011315640062093735, ..."
2,I think Agent[00] is a werewolf,ESTIMATE,Agent[00],WEREWOLF,None,"[-0.029915498569607735, -0.0012816088274121284..."
3,I think Agent[00] is a possessed,ESTIMATE,Agent[00],POSSESSED,None,"[-0.02937072515487671, 0.018204612657427788, -..."
4,I think Agent[01] is a villeger,ESTIMATE,Agent[01],VILLEGER,None,"[-0.03308890759944916, 0.0016154177719727159, ..."
...,...,...,...,...,...,...
344,What Agent[00] said is wrong,NOT AGREE,Agent[00],None,None,"[-0.03842293843626976, 0.019114114344120026, 0..."
345,What Agent[01] said is wrong,NOT AGREE,Agent[01],None,None,"[-0.03957590460777283, 0.01657181605696678, 0...."
346,What Agent[02] said is wrong,NOT AGREE,Agent[02],None,None,"[-0.032267309725284576, 0.016541307792067528, ..."
347,What Agent[03] said is wrong,NOT AGREE,Agent[03],None,None,"[-0.036613430827856064, 0.01735736057162285, 0..."
